Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "SIDDHARTH YASHASWEE"
COLLABORATORS = ""

---

# Distance Measures
In this notebook we will implement the approximate computation of geodesic distances via the heat method. We will also compute the Chamfer distance between two point clouds.

In [2]:
from ipywidgets import interact, fixed
import numpy as np
import scipy as sp
from scipy.sparse import csr_matrix, diags
from scipy.sparse.linalg import spsolve, lsqr
from scipy.spatial.distance import cdist
import openmesh as om
import k3d

In [3]:
mesh = om.read_trimesh("spot.obj")
bob_mesh = om.read_trimesh("bob.obj")
A = sp.sparse.load_npz("A.npz")
W = -sp.sparse.load_npz("W.npz")

## Heat Method
For the heat method you will have to compute the heat distribution. Then you have to derive gradients for this distribution on the triangle mesh and then solve a Poisson equation to compute a geodesic distance field.

In order to compute the heat distribution with `compute_heat_distribution` you will need the (uninverted) mass matrix `A` and the weak Laplacian Beltrami Operator `W` for the spot mesh (stored in `mesh`). We have precomputed and loaded those matrices for you above. Furthermore, we require a time parameter `t` (in our case `t=1`) and an intial heat distribution for each vertex of the mesh. Here we apply heat to a single point (the vertex with id 1570) for which we want to compute the geodesic distance field. Since `A` and `W` are sparse matrices use `spsolve` to solve the linear equation system. The result should be a numpy array with one axis with scalar values for every vertex of the mesh. The heat distribution should then look like [this](./heat_distribution.html) (both the intial heat distribution and the one at time step `t` are shown).

Next, we need to compute the gradient of the heat distribution on our mesh with `compute_gradient_on_trimesh`. Here `mesh` is the spot mesh and `u` the heat distribution we computed above (per vertex). The result should be a 3D gradient vector per triangle (again as a numpy array). You can use the `compute_triangle_area` and `compute_normal` helper functions per triangle (`a`, `b`, and `c` are the 3 3D points of the triangle). In order to iterate over the vertices of each face you can either use `mesh.fv(fh)` for a particular face handle `fh` or get all face vertex indices with `mesh.fv_indices()` as a numpy array. Afterwards you will have to transform the gradient to a vector field that matches the gradient of a distance field with `compute_gradient_of_distance_field`. This function takes as input a numpy array with 3D gradients per triangle face and returns a vector field as a numpy array with 3D vectors per face. If you implemented everything correctly the resulting vector field should look like [this](./gradient_field.html).

Now that we have the gradient field for our distance field, we need to recover the distance field in question. To this end we have to compute the divergence of that gradient field in order to setup and solve a Poisson equation. Implement the computation of the divergence with `compute_divergence_on_trimesh` that takes as as input the spot mesh `mesh` and the vector field computed above with `compute_gradient_of_distance_field` as `vec`. It returns a numpy array with a scalar per vertex. You can use `mesh.vertices()` to iterate over all vertices of the mesh. Use `mesh.voh(vh)` to iterate over all outgoing halfedges for a vertex handle `vh`. You can get the corresponding (unique) face handle for a halfedge handle `hh` with `mesh.face_handle(hh)`. To jump to the next halfedge handle from a halfedge handle `hh` in a face use `mesh.next_halfedge_handle(hh)`. The vertex handle that a halfedge `hh` points to can be looked up with `mesh.to_vertex_handle(hh)`. The cotanget for two vectors $v_1$ and $v_2$ can be computed as $\frac{v_1^Tv_2}{||v_1 \times v_2||}$. If you implemented everything correctly your divergence should look like [this](./divergence.html). You can now setup and solve the Poisson equation in `compute_distance_field` that takes as input the weak Laplace Beltrami Operator and the divergence field. It returns the distance field (a numpy array with scalars per vertex). Use `lsqr` to solve the Poisson equation since it is over-determined. Your solution should look like [this](./distance_field.html).
Note that the distance field can have negative values, as the solution of the Poisson equation is invariant to a constant offset.

In [4]:
def compute_triangle_area(a, b, c):
    return np.linalg.norm(np.cross(b-a, c-a))/2

def compute_normal(a, b, c):
    n = np.cross(b-a, c-a)
    n /= np.linalg.norm(n)
    return n

def compute_gradient_on_trimesh(mesh, u):
    ### YOUR CODE HERE
    points = mesh.points()
    faces = mesh.fv_indices()
    gradients = []
    
    for face in faces:
        area = compute_triangle_area(points[face[0]], points[face[1]], points[face[2]])
        normal = compute_normal(points[face[0]],points[face[1]], points[face[2]])
        
        e0 = points[face[2]]-points[face[1]]
        e1 = points[face[0]]-points[face[2]]
        e2 = points[face[1]]-points[face[0]]
        
        gradient = u[face[0]]*np.cross(normal, e0) + u[face[1]]*np.cross(normal, e1) + u[face[2]]*np.cross(normal, e2)
        gradient /= 2*area
        
        gradients.append(gradient)
    
    ngrads=np.stack(gradients, axis=0)
    return ngrads

def compute_divergence_on_trimesh(mesh, vec):
    ### YOUR CODE HERE

    #store scalar per vertex
    divergence = []
    
    #iterate over all vertices
    for vh in mesh.vertices():
        
        #position of the vertex
        ipos = mesh.point(vh)
        
        vertex_area = 0
        vertex_div = 0
        
        #iterate over all outgoing halfedges
        for oh in mesh.voh(vh):
            
            #face handle for halfedge
            fh = mesh.face_handle(oh)
            Xj = vec[fh.idx()]
            
            jpos1 = mesh.point(mesh.to_vertex_handle(oh))
            jpos2 = mesh.point(mesh.to_vertex_handle(mesh.next_halfedge_handle(oh)))
            
            e1X = np.dot((jpos1 - ipos),Xj)
            e2X = np.dot((jpos2 - ipos),Xj)
            
            #cot1
            cot1 = np.dot(ipos - jpos2, jpos1 - jpos2)/np.linalg.norm(np.cross(ipos - jpos2, jpos1 - jpos2))
            
            #cot2
            cot2 = np.dot(ipos - jpos1, jpos2 - jpos1)/np.linalg.norm(np.cross(ipos - jpos1, jpos2 - jpos1))
            
            vertex_div += (cot1*e1X + cot2*e2X)
          
            vertex_area += compute_triangle_area(ipos,jpos1,jpos2)
        
        #do not dide by area, cancelled out in poissons equation
        vertex_div = vertex_div/2
        
        divergence.append(vertex_div)
    #print(divergence[0])
    #print(divergence[1000])

    return np.array(divergence)
    
def compute_heat_distribution(A, W, t, init_heat):
    ### YOUR CODE HERE   
    return spsolve(A-(t*W), A*init_heat)

def compute_gradient_of_distance_field(grads):
    ### YOUR CODE HERE
    vec_sums = np.sum(grads**2, axis=1)
    vec_sums = vec_sums[:,None]
    ngrads = (-1)*grads / np.sqrt(vec_sums)
    return ngrads

def compute_distance_field(W, div):
    ### YOUR CODE HERE
    return lsqr(W, div)[0]

In [5]:
init_u = np.zeros(mesh.n_vertices())
init_u[1570] = 1.
t = 1
u_t = compute_heat_distribution(A, W, t, init_u)
np.testing.assert_approx_equal(u_t[0], 0.00012, significant=2)
np.testing.assert_approx_equal(u_t[1000], 0.00017, significant=2)
np.testing.assert_approx_equal(u_t[-1], 0.000093, significant=2)

In [6]:
plot = k3d.plot()
plot += k3d.mesh(mesh.points(), mesh.fv_indices(), attribute=init_u, color_map=k3d.matplotlib_color_maps.viridis)
plot += k3d.mesh(mesh.points() + np.array([-1.,0,0]), mesh.fv_indices(), attribute=u_t, color_map=k3d.matplotlib_color_maps.viridis)
plot

C:\Users\lenovo1\.conda\envs\shapeanalysis\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
C:\Users\lenovo1\.conda\envs\shapeanalysis\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "int32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [7]:
u_t_grad = compute_gradient_on_trimesh(mesh, u_t)
X = compute_gradient_of_distance_field(u_t_grad)
np.testing.assert_approx_equal(X[0,0], -0.59, significant=2)
np.testing.assert_approx_equal(X[0,1], -0.38, significant=2)
np.testing.assert_approx_equal(X[0,2], 0.71, significant=2)
np.testing.assert_approx_equal(X[-1,0], 0.06, significant=2)
np.testing.assert_approx_equal(X[-1,1], -0.89, significant=2)
np.testing.assert_approx_equal(X[-1,2], -0.46, significant=2)
np.testing.assert_approx_equal(X[1000,0], -0.96, significant=2)
np.testing.assert_approx_equal(X[1000,1], -0.23, significant=2)
np.testing.assert_approx_equal(X[1000,2], -0.19, significant=2)

In [8]:
plot = k3d.plot()
plot += k3d.mesh(mesh.points(), mesh.fv_indices(), attribute=u_t, color_map=k3d.matplotlib_color_maps.viridis)
plot += k3d.vectors(mesh.points()[mesh.fv_indices()].mean(1), X*0.05, use_head=False)
plot

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [9]:
div = compute_divergence_on_trimesh(mesh, X)
phi = compute_distance_field(W, div)
phi -= phi.min() # solution is not unique up to constant factor. Therefore set minimum to zero
np.testing.assert_approx_equal(div[0], 0.0096, significant=2)
np.testing.assert_approx_equal(div[1000], 0.0056, significant=2)
np.testing.assert_approx_equal(phi[0], 1.41, significant=2)
np.testing.assert_approx_equal(phi[1000], 1.09, significant=2)

In [10]:
k3d.mesh(mesh.points(), mesh.fv_indices(), attribute=div, color_map=k3d.matplotlib_color_maps.viridis)

Output()

In [11]:
k3d.mesh(mesh.points(), mesh.fv_indices(), attribute=phi, color_map=k3d.matplotlib_color_maps.coolwarm)

Output()

In [12]:
def plot_geodesics_heat(A,W,vertex):
    init_u = np.zeros(mesh.n_vertices())
    init_u[vertex] = 1.
    plot = k3d.plot()
    u_t = compute_heat_distribution(A, W, 1, init_u)
    u_t_grad = compute_gradient_on_trimesh(mesh, u_t)
    X = compute_gradient_of_distance_field(u_t_grad)
    div = compute_divergence_on_trimesh(mesh, X)
    phi = compute_distance_field(W, div)
    plot += k3d.mesh(mesh.points(), mesh.fv_indices(), attribute=phi, color_map=k3d.matplotlib_color_maps.coolwarm)
    plot.display()

interact(plot_geodesics_heat,
         A = fixed(A),
         W = fixed(W),
         vertex = range(mesh.n_vertices()),
         continous_update=False)

C:\Users\lenovo1\.conda\envs\shapeanalysis\lib\site-packages\traitlets\traitlets.py:711: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  silent = bool(old_value == new_value)
C:\Users\lenovo1\.conda\envs\shapeanalysis\lib\site-packages\traitlets\traitlets.py:1738: SparseEfficiencyWarning: Comparing sparse matrices using == is inefficient, try using != instead.
  getattr(cls, name).set(self, value)


interactive(children=(Dropdown(description='vertex', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14…

<function __main__.plot_geodesics_heat(A, W, vertex)>

## Chamfer Distance
Compute the Chamfer distance with `chamfer_distance` for the two point clouds below. You can use `cdist` to compute the pairwise (Euclidean) distance matrix between the two point clouds.

In [13]:
plot = k3d.plot()
plot += k3d.points(mesh.points(), point_size=0.02, color=0x0000ff)
plot += k3d.points(bob_mesh.points(), point_size=0.02, color=0xff0000)
plot

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [14]:
def chamfer_distance(pts_A, pts_B):
    ### YOUR CODE HERE
    n_a = pts_A.shape[0]
    m_b = pts_B.shape[0]
    
    matrix = cdist(pts_A, pts_B)
    
    dista=0.0
    distb=0.0
    
    for x in range(n_a):
        dxY = np.min(matrix[x,:])
        dista += dxY
    
    for y in range(m_b):
        dyX = np.min(matrix[:,y])
        distb += dyX

    return dista/n_a+distb/m_b
    

In [15]:
pts_A = mesh.points()
pts_B = bob_mesh.points()
d = chamfer_distance(pts_A, pts_B)
np.testing.assert_approx_equal(d, 0.51, significant=2)